# ICE-NMF Evaluation ml1m

In [1]:
#imports
import numpy as np 
import csv
import re
import sys
import math
import matplotlib.pyplot as plt
from numpy import linalg as L
import pandas as pd
import random
from sklearn.preprocessing import normalize
from numba import njit
from collections import defaultdict
import random
from implicit_data import ImplicitData
import pandas as pd
from BISGD import BISGD
from ICE_NMF import ICE_NMF
from collections import defaultdict
from EvalPrequential import EvalPrequential
from datetime import datetime
import getopt

## Functions


In [2]:
def sigmoid(x):
  return 1 / (1 + math.exp(-x))

def sumvector(V,x):
    result = []
    for v in V:
        temp = v + x
        result.append(temp)
    return result    

def multvector(x,V):
    result = []
    for v in V:
        temp = v * x
        result.append(temp)
    return result    

def TransformVec(V):
    for i in range(len(V)):
        V[i] = sigmoid(V[i])
    D = LA.norm(V)
    D=1/D
    new = D * V
  

    return new


## Get the data

In [3]:
sys.path.append('./')
configPath = 'config'
dataPath = 'ml1m_gte5.csv'


Data = []
data = pd.read_csv("ml1m_gte5.csv","\t")
print(data)
Data = data.values.tolist()



        UserID  ItemID   Timestamp
0         6040     593   956703954
1         6040    2019   956703977
2         6040     213   956704056
3         6040    3111   956704056
4         6040     912   956704191
...        ...     ...         ...
226305    5950    3363  1046367948
226306    5950     111  1046368241
226307    5950    3196  1046368462
226308    5948    1180  1046437879
226309    4958    1407  1046454443

[226310 rows x 3 columns]


# Full dataset

In [6]:
Data_Transpose = np.transpose(Data)
stream = ImplicitData(Data_Transpose[0],Data_Transpose[1])



num_clusters = {8,16,24}
overlap = 0.7
cl_num_iterations = 10
cl_learn_rate =0.1
cl_regularization = 0.1
num_factors = 160
num_iter = 8
learn_rate = 0.1
regularization = 0.4


In [ ]:
%%capture out

res = {}

for num in num_clusters:
    print('')
    print('num_clusters: ', num)
    model = ICE_NMF(ImplicitData([],[]), num, overlap, cl_num_iterations, cl_learn_rate, cl_regularization, 
        num_factors, num_iter, learn_rate, regularization, random_seed = 100)
    eval = EvalPrequential(model,stream, metrics = ["Recall@20"])

    start_recommend = datetime.now()
    print('start time', start_recommend)

    results=eval.EvaluateTime(0,stream.size, 1)
    res[num] = np.mean(results['Recall@20'])
    
    print('npmean(resuls[Recall@20])', res[num])
    end_recommend = datetime.now()
    print('end time', end_recommend)

    tempo = end_recommend - start_recommend

    print('run time', tempo)
    print('recommend',np.mean(results['time_recommend']))
    print('update',np.mean(results['time_update'])) 

In [6]:
print(out)

start time 2023-10-20 17:14:17.968296
npmean(resuls[Recall@20]) 0.06570472439204453
end time 2023-10-20 23:54:29.235004
run time 6:40:11.266708

get tuple 4.492414534410635e-06
recommend 0.10413758091748473
eval_point 7.329267204837828e-05
update 0.0019220208559132756



## Sampled datasets

### ml1m_sample5_1

In [4]:
sys.path.append('./')
configPath = 'config'

Data = []
data = pd.read_csv("ml1m_sample5_1.csv")
print(data)
Data = data.values.tolist()

Data = Data[1:]
 

       Unnamed: 0  UserID  ItemID   Timestamp
0             365    6029     899   956721639
1             366    6029    3548   956721700
2             367    6029    3545   956721732
3             368    6029    3543   956721759
4             369    6029    3549   956721780
...           ...     ...     ...         ...
11252      225903     937    2300  1042909231
11253      225935     195    3746  1043562457
11254      226240    4312     750  1045841700
11255      226241    4312    1267  1045841716
11256      226267     911    2409  1046066544

[11257 rows x 4 columns]


In [6]:
Data_Transpose = np.transpose(Data)
stream = ImplicitData(Data_Transpose[1],Data_Transpose[2])



num_clusters = 8
cl_num_iterations = 20
cl_learn_rate =0.1
cl_regularization = 0.1
num_factors = 160
num_iter = 8
learn_rate = 0.1
regularization = 0.4
model = ICE_NMF(ImplicitData([],[]), num_clusters, cl_num_iterations, cl_learn_rate, cl_regularization, 
    num_factors, num_iter, learn_rate, regularization, random_seed = 100)
eval = EvalPrequential(model,stream, metrics = ["Recall@20"])
start_recommend = datetime.now()
print('start time', start_recommend)

results=eval.EvaluateTime(0,stream.size, 10)

print('npmean(resuls[Recall@20])', np.mean(results['Recall@20']))
end_recommend = datetime.now()
print('end time', end_recommend)

tempo = end_recommend - start_recommend

print('run time', tempo)
print('')
print('get tuple',np.mean(results['time_get_tuple']))
print('recommend',np.mean(results['time_recommend']))
print('eval_point',np.mean(results['time_eval_point']))
print('update',np.mean(results['time_update'])) 

start time 2022-09-19 23:54:08.876029
npmean(resuls[Recall@20]) 0.08955223880597014
end time 2022-09-19 23:54:58.153788
run time 0:00:49.277759

get tuple 2.5253390791404306e-06
recommend 0.020040606908153503
eval_point 1.707445434355966e-05
update 0.002342325050778311


### ml1m_sample5_2

In [7]:
sys.path.append('./')
configPath = 'config'

Data = []
data = pd.read_csv("ml1m_sample5_2.csv")
print(data)
Data = data.values.tolist()

Data = Data[1:]
 

       Unnamed: 0  UserID  ItemID   Timestamp
0             206    6033    1193   956713500
1             207    6033    1214   956713838
2             208    6033    2028   956713838
3             209    6033     110   956713865
4             210    6033    1036   956713898
...           ...     ...     ...         ...
11037      226258    5329    1212  1045895993
11038      226259    5329    2692  1045896006
11039      226260    5329     942  1045896053
11040      226261    5329    2019  1045896053
11041      226262    5329    3035  1045896063

[11042 rows x 4 columns]


In [8]:
Data_Transpose = np.transpose(Data)
stream = ImplicitData(Data_Transpose[1],Data_Transpose[2])



num_clusters = 8
cl_num_iterations = 20
cl_learn_rate =0.1
cl_regularization = 0.1
num_factors = 160
num_iter = 8
learn_rate = 0.1
regularization = 0.4
model = ICE_NMF(ImplicitData([],[]), num_clusters, cl_num_iterations, cl_learn_rate, cl_regularization, 
    num_factors, num_iter, learn_rate, regularization, random_seed = 100)
eval = EvalPrequential(model,stream, metrics = ["Recall@20"])
start_recommend = datetime.now()
print('start time', start_recommend)

results=eval.EvaluateTime(0,stream.size, 10)

print('npmean(resuls[Recall@20])', np.mean(results['Recall@20']))
end_recommend = datetime.now()
print('end time', end_recommend)

tempo = end_recommend - start_recommend

print('run time', tempo)
print('')
print('get tuple',np.mean(results['time_get_tuple']))
print('recommend',np.mean(results['time_recommend']))
print('eval_point',np.mean(results['time_eval_point']))
print('update',np.mean(results['time_update'])) 

start time 2022-09-20 00:00:59.102854
npmean(resuls[Recall@20]) 0.08310749774164408
end time 2022-09-20 00:01:49.000006
run time 0:00:49.897152

get tuple 2.5987862655298293e-06
recommend 0.02167174163549573
eval_point 1.8516099442203733e-05
update 0.002338107713440732


### ml1m_sample5_3

In [9]:
sys.path.append('./')
configPath = 'config'

Data = []
data = pd.read_csv("ml1m_sample5_3.csv")
print(data)
Data = data.values.tolist()

Data = Data[1:]

       Unnamed: 0  UserID  ItemID   Timestamp
0             110    6035    1193   956710879
1             120    6035    2628   956711024
2             123    6035    3175   956711064
3             124    6035    2571   956711064
4             129    6035    1527   956711121
...           ...     ...     ...         ...
11544      226173    5654    1500  1045160087
11545      226174    5654      39  1045160173
11546      226175    5654    3358  1045160206
11547      226176    5654    3526  1045162240
11548      226203    5654    2022  1045669508

[11549 rows x 4 columns]


In [10]:
Data_Transpose = np.transpose(Data)
stream = ImplicitData(Data_Transpose[1],Data_Transpose[2])



num_clusters = 8
cl_num_iterations = 20
cl_learn_rate =0.1
cl_regularization = 0.1
num_factors = 160
num_iter = 8
learn_rate = 0.1
regularization = 0.4
model = ICE_NMF(ImplicitData([],[]), num_clusters, cl_num_iterations, cl_learn_rate, cl_regularization, 
    num_factors, num_iter, learn_rate, regularization, random_seed = 100)
eval = EvalPrequential(model,stream, metrics = ["Recall@20"])
start_recommend = datetime.now()
print('start time', start_recommend)

results=eval.EvaluateTime(0,stream.size, 10)

print('npmean(resuls[Recall@20])', np.mean(results['Recall@20']))
end_recommend = datetime.now()
print('end time', end_recommend)

tempo = end_recommend - start_recommend

print('run time', tempo)
print('')
print('get tuple',np.mean(results['time_get_tuple']))
print('recommend',np.mean(results['time_recommend']))
print('eval_point',np.mean(results['time_eval_point']))
print('update',np.mean(results['time_update'])) 

start time 2022-09-20 00:02:59.329939
npmean(resuls[Recall@20]) 0.08608695652173913
end time 2022-09-20 00:03:50.716054
run time 0:00:51.386115

get tuple 2.5845333742855904e-06
recommend 0.021488810000212296
eval_point 1.7172357310419497e-05
update 0.002302042910249166


### ml1m_sample5_4

In [11]:
sys.path.append('./')
configPath = 'config'

Data = []
data = pd.read_csv("ml1m_sample5_4.csv")
print(data)
Data = data.values.tolist()

Data = Data[1:]

       Unnamed: 0  UserID  ItemID   Timestamp
0              42    6038    1079   956707547
1              43    6038    1148   956707604
2              44    6038    3088   956707640
3              45    6038    1223   956707734
4             206    6033    1193   956713500
...           ...     ...     ...         ...
11461      226168    4958    2087  1045000073
11462      226242     839     909  1045857499
11463      226243     839     750  1045857818
11464      226244     839     954  1045859395
11465      226310    4958    1407  1046454443

[11466 rows x 4 columns]


In [12]:
Data_Transpose = np.transpose(Data)
stream = ImplicitData(Data_Transpose[1],Data_Transpose[2])



num_clusters = 8
cl_num_iterations = 20
cl_learn_rate =0.1
cl_regularization = 0.1
num_factors = 160
num_iter = 8
learn_rate = 0.1
regularization = 0.4
model = ICE_NMF(ImplicitData([],[]), num_clusters, cl_num_iterations, cl_learn_rate, cl_regularization, 
    num_factors, num_iter, learn_rate, regularization, random_seed = 100)
eval = EvalPrequential(model,stream, metrics = ["Recall@20"])
start_recommend = datetime.now()
print('start time', start_recommend)

results=eval.EvaluateTime(0,stream.size, 10)

print('npmean(resuls[Recall@20])', np.mean(results['Recall@20']))
end_recommend = datetime.now()
print('end time', end_recommend)

tempo = end_recommend - start_recommend

print('run time', tempo)
print('')
print('get tuple',np.mean(results['time_get_tuple']))
print('recommend',np.mean(results['time_recommend']))
print('eval_point',np.mean(results['time_eval_point']))
print('update',np.mean(results['time_update'])) 

start time 2022-09-20 00:05:35.472921
npmean(resuls[Recall@20]) 0.09075043630017451
end time 2022-09-20 00:06:23.909959
run time 0:00:48.437038

get tuple 2.4059793078842197e-06
recommend 0.019503940462441968
eval_point 1.6742975924027528e-05
update 0.0022677946236265628


### ml1m_sample5_5

In [13]:
sys.path.append('./')
configPath = 'config'

Data = []
data = pd.read_csv("ml1m_sample5_5.csv")
print(data)
Data = data.values.tolist()

Data = Data[1:]

       Unnamed: 0  UserID  ItemID   Timestamp
0               1    6040     593   956703954
1               2    6040    2019   956703977
2               3    6040     213   956704056
3               4    6040    3111   956704056
4               5    6040     912   956704191
...           ...     ...     ...         ...
11050      226029     424    1935  1043844911
11051      226054     424    1939  1044296801
11052      226201     424    3739  1045581875
11053      226239     424    2494  1045837566
11054      226279     424    3535  1046112575

[11055 rows x 4 columns]


In [14]:
Data_Transpose = np.transpose(Data)
stream = ImplicitData(Data_Transpose[1],Data_Transpose[2])



num_clusters = 8
cl_num_iterations = 20
cl_learn_rate =0.1
cl_regularization = 0.1
num_factors = 160
num_iter = 8
learn_rate = 0.1
regularization = 0.4
model = ICE_NMF(ImplicitData([],[]), num_clusters, cl_num_iterations, cl_learn_rate, cl_regularization, 
    num_factors, num_iter, learn_rate, regularization, random_seed = 100)
eval = EvalPrequential(model,stream, metrics = ["Recall@20"])
start_recommend = datetime.now()
print('start time', start_recommend)

results=eval.EvaluateTime(0,stream.size, 10)

print('npmean(resuls[Recall@20])', np.mean(results['Recall@20']))
end_recommend = datetime.now()
print('end time', end_recommend)

tempo = end_recommend - start_recommend

print('run time', tempo)
print('')
print('get tuple',np.mean(results['time_get_tuple']))
print('recommend',np.mean(results['time_recommend']))
print('eval_point',np.mean(results['time_eval_point']))
print('update',np.mean(results['time_update'])) 

start time 2022-09-20 00:08:02.496520
npmean(resuls[Recall@20]) 0.09202453987730061
end time 2022-09-20 00:08:49.424471
run time 0:00:46.927951

get tuple 2.475377554399143e-06
recommend 0.01944007443086188
eval_point 1.6784375430616134e-05
update 0.0022310630223063895
